imports

In [1]:
import numpy as np
import pandas as pd
from tg.grammar_ru.common import Loc
from dotenv import load_dotenv
load_dotenv(Loc.root_path / 'environment.env')
import re

from tg.common import DataBundle
from tg.common.ml.batched_training import train_display_test_split
from tg.grammar_ru.features import PyMorphyFeaturizer

from tg.grammar_ru.corpus import ITransfuseSelector
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem


new = {'ая', 'ого', 'ое', 'ой', 'ом', 'ому',
       'ую', 'ые', 'ый', 'ым', 'ыми', 'ых'} # тут нет окнчаний превосходных форм и ою

# полнейшей, наипрочнейшего, важнейшие,меньшим, милейший, наистраннейшее, новейших, малейшем, слабейшему, меньшими
good = {'ая', 'его', 'ее', 'ей', 'ем', 'ему',
        'ие', 'ий', 'им', 'ими', 'их', 'ую', 'яя', 'юю'}
#  'яя', 'юю' - последняя, последнюю

big = {'ая', 'ие', 'им', 'ими', 'их', 'ого',
       'ое', 'ой', 'ом', 'ому', 'ою', 'ую'}

POSSIBLE_ENDINGS = set().union(new, good, big)
endings_nums = {e: i for i, e in enumerate(
    sorted(list(POSSIBLE_ENDINGS)))}
num_by_ending = endings_nums
ending_by_num = {v:k for k, v in endings_nums.items()}

new_declination_labels = {num for e, num in endings_nums.items() if e in new}
good_declination_labels = {num for e, num in endings_nums.items() if e in good}
big_declination_labels = {num for e, num in endings_nums.items() if e in big}




In [8]:
good_declination_labels

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 17, 23, 24}

In [2]:
from tg.common import DataBundle
from tg.common.ml.batched_training import IndexedDataBundle
from tg.grammar_ru.components.plain_context_builder import PlainContextBuilder
bundle_1_declination_path = Loc.data_cache_path/'bundles/agreement/mid50_1_declination'
bundle_2_declination_path = Loc.data_cache_path/'bundles/agreement/mid50_2_declination'
# bundle_full_0_declination_path = Loc.data_cache_path/'bundles/agreement/full_mystemless_0_declination'


In [3]:
# db = DataBundle.load(Loc.data_cache_path/'bundles/agreement/mid50')
# ids_2_type=set(db.src[db.src.declension_type==2].word_id)
# db['index'] = db.index[db.index.word_id.isin(ids_2_type) & db.index.label.isin(big_declination_labels)]
# db = db.copy()
# db.save(bundle_2_declination_path)

In [4]:
db = DataBundle.load(Loc.data_cache_path/'bundles/agreement/mid50')
ids_1_type=set(db.src[db.src.declension_type==1].word_id)
db['index'] = db.index[db.index.word_id.isin(ids_1_type) & db.index.label.isin(good_declination_labels)]
db = db.copy()
db.save(bundle_1_declination_path)

In [12]:
# db2 = DataBundle.load(bundle_2_declination_path)
# db2.src

In [14]:
ids_1_type=db.src[db.src.declension_type==1]
db.src[db.src.word_id.isin(db.index[db.index.label.isin([23, 24])].word_id)].word

719          последняя
10989        последнюю
11450           заднюю
13351        последнюю
16397         домашнюю
              ...     
2465177    многолетняя
2465614         нижнюю
2467733       домашнюю
2483982     внутренняя
2488018          Синяя
Name: word, Length: 514, dtype: object

Отправка

In [6]:
project_name = 'agreementproject'
dataset_name = 'agreement_adj_mid50_1_declination'#tiny_0_declination
# dataset_name = 'agreement_adj_mid50'
# dataset_name = 'agreement_adj_mid50_2_declination'#tiny_0_declination
# dataset_name = 'agreement_adj_mid50_all_decl_masked'#tiny_0_declination
# dataset_name = 'agreement_adj_tiny_all_decl_masked'#tiny_0_declination
# dataset_name = 'agreement_adj_debug4labels'
# dataset_name = 'agreement_adj_toy'

bucket = 'agreementadjbucket'

In [7]:
from tg.grammar_ru.components.yandex_storage.s3_yandex_helpers import S3YandexHandler


In [8]:
s3path = f'datasphere/{project_name}/datasets/{dataset_name}'
S3YandexHandler.upload_folder(bucket, s3path, Loc.data_cache_path/f'bundles/agreement/mid50_1_declination')